In [1]:
import pandas as pd
import numpy as np
import us
import os
import subprocess
from pathlib import Path
import plotly.express as px

In [2]:
# assumes git is installed
# assumes you are storing data in parent directory

os.chdir('../data/covidclinicaldata')
cmd = ['git', 'pull', 'https://github.com/mdcollab/covidclinicaldata.git']
out = subprocess.run(cmd, stdout=subprocess.PIPE)
print(out.stdout.decode())

data_path = Path().joinpath('data')

df = pd.DataFrame([])
for csv in data_path.glob('*.csv'):
    dft = pd.read_csv(csv)
    df = pd.concat([df, dft])

os.chdir('../../scripts')

Already up to date.



In [3]:
df = df.rename(columns={'temperature': 'temperature_C'})
df['temperature_F'] = df['temperature_C'] * 9 / 5 + 32
num_cols = []
bool_cols = ['high_risk_exposure_occupation',
       'high_risk_interactions', 'diabetes', 'chd', 'htn', 'cancer', 'asthma',
       'copd', 'autoimmune_dis', 'ctab', 'labored_respiration', 'rhonchi',
       'wheezes', 'cough', 'fever', 'sob',
       'diarrhea', 'fatigue', 'headache', 'loss_of_smell', 'loss_of_taste',
       'runny_nose', 'muscle_sore', 'sore_throat']


In [4]:
fig = px.histogram(
    data_frame=df,
    x='temperature_F',
    color='covid19_test_results',
    marginal='rug',
    hover_data=['age', 'high_risk_exposure_occupation',
       'high_risk_interactions', 'diabetes', 'chd', 'htn', 'cancer', 'asthma',
       'copd', 'autoimmune_dis'],
    title='COVID-19 Temperature Histogram<br>'
          'Source:<a href="https://github.com/mdcollab/covidclinicaldata">'
          'Coronavirus Disease 2019 (COVID-19) Clinical Data Repository</a>'
    )
fig.write_html('../charts/clinical_temperature_histogram.html')

In [5]:
fig = px.bar(
    data_frame=df,
    x='diarrhea',
    color='covid19_test_results',
    hover_data=['age', 'high_risk_exposure_occupation',
       'high_risk_interactions', 'diabetes', 'chd', 'htn', 'cancer', 'asthma',
       'copd', 'autoimmune_dis'],
    title='COVID-19 Diarrhea<br>'
          'Source:<a href="https://github.com/mdcollab/covidclinicaldata">'
          'Coronavirus Disease 2019 (COVID-19) Clinical Data Repository</a>'
    )
# fig.write_html('../lyteworx.github.io/charts/covid19_diarrhea_bar.html')

In [6]:
corr_cols = ['age', 'temperature_F', 'pulse', 'sys', 'dia', 'rr',
       'sats', 'days_since_symptom_onset']
corr = df[corr_cols].corr()
fig = px.imshow(
    corr,
    x=corr_cols,
    y=corr_cols,
    color_continuous_scale=px.colors.diverging.Picnic,
    color_continuous_midpoint=0

    )
# fig.write_html('../charts/numerical_symptom_correlation_matrix.html')


In [7]:
for c in bool_cols:
    idx = df[df[c] == True].index
    df.at[idx, c] = 1
    idx = df[df[c] == False].index
    df.at[idx, c] = -1
    df[c] = df[c].fillna(0)

bool_corr_pos = df[bool_cols][df.covid19_test_results == 'Positive'].corr()
bool_corr_pos = pd.melt(bool_corr_pos.reset_index(), id_vars='index')
bool_corr_pos.columns = ['x', 'y', 'value']

bool_corr_neg = df[bool_cols][df.covid19_test_results == 'Negative'].corr()
bool_corr_neg = pd.melt(bool_corr_neg.reset_index(), id_vars='index')
bool_corr_neg.columns = ['x', 'y', 'value']
bool_corr_pos

,x,y,value
0,high_risk_exposure_occupation,high_risk_exposure_occupation,1.000000
1,high_risk_interactions,high_risk_exposure_occupation,0.304078
2,diabetes,high_risk_exposure_occupation,-0.115168
3,chd,high_risk_exposure_occupation,0.060971
4,htn,high_risk_exposure_occupation,-0.072936
...,...,...,...
571,loss_of_smell,sore_throat,0.398251
572,loss_of_taste,sore_throat,0.473866
573,runny_nose,sore_throat,0.557676
574,muscle_sore,sore_throat,0.489507


In [8]:
fig = px.scatter(
    bool_corr_pos,
    x='x',
    y=bool_corr_pos['y'],
    size=bool_corr_pos['value'].fillna(0).abs() * 500,
    symbol_sequence=['square'],
    color=bool_corr_pos['value'].abs(),
    title='COVID-19 Symptom Correlation for testing <em>POSITIVE</em>.<br>'
          'Source:<a href="https://github.com/mdcollab/covidclinicaldata">'
          'Coronavirus Disease 2019 (COVID-19) Clinical Data Repository</a>',
    )
fig.update_xaxes(showgrid=True, ticks='outside', tickson='boundaries',
                 scaleratio=1, scaleanchor='y', title_text='symptoms',
                 tickangle=30)
fig.update_yaxes(showgrid=True, ticks='outside', tickson='boundaries',
                 scaleratio=1, title_text='symptoms')
fig.write_html('../charts/covid-19_positive_symptom_correlation_matrix.html')


In [9]:
fig = px.scatter(
    bool_corr_neg,
    x='x',
    y=bool_corr_neg['y'],
    size=bool_corr_neg['value'].fillna(0).abs() * 500,
    symbol_sequence=['square'],
    color=bool_corr_neg['value'].abs(),
    title='COVID-19 Symptom Correlation for testing <em>NEGATIVE</em>.<br>'
          'Source:<a href="https://github.com/mdcollab/covidclinicaldata">'
          'Coronavirus Disease 2019 (COVID-19) Clinical Data Repository</a>',
    )
fig.update_xaxes(showgrid=True, ticks='outside', tickson='boundaries',
                 scaleratio=1, scaleanchor='y', title_text='symptoms',
                 tickangle=30)
fig.update_yaxes(showgrid=True, ticks='outside', tickson='boundaries',
                 scaleratio=1, title_text='symptoms')
fig.write_html('../charts/covid-19_negative_symptom_correlation_matrix.html')